In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3

%matplotlib inline

In [7]:
def get_dataframe_from_sql(query, db_path = '../../Downloads/database.sqlite'):
    # establish a connection to the database
    conn = sqlite3.connect(db_path)

    # load the query results into a pandas dataframe
    df = pd.read_sql_query(query, conn)

    # close the connection to the database
    conn.close()

    # return the dataframe
    return df

In [8]:
# define your SQL query
query = "SELECT name FROM sqlite_master WHERE type='table';"

# load the query results into a pandas dataframe
get_dataframe_from_sql(query)

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


In [10]:
# define your SQL query
query = "PRAGMA table_info(Player_Attributes);"

# load the query results into a pandas dataframe
get_dataframe_from_sql(query)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,player_fifa_api_id,INTEGER,0,None,0
2,2,player_api_id,INTEGER,0,None,0
3,3,date,TEXT,0,None,0
4,4,overall_rating,INTEGER,0,None,0
5,5,potential,INTEGER,0,None,0
6,6,preferred_foot,TEXT,0,None,0
7,7,attacking_work_rate,TEXT,0,None,0
8,8,defensive_work_rate,TEXT,0,None,0
9,9,crossing,INTEGER,0,None,0
